In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.7


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1


In [ ]:
!sudo update-alternatives --config python3


There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.11   2         auto mode
  1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.11   2         manual mode
  3            /usr/bin/python3.7    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 3
update-alternatives: using /usr/bin/python3.7 to provide /usr/bin/python3 (python3) in manual mode


In [ ]:
!curl https://bootstrap.pypa.io/pip/3.7/get-pip.py -o get-pip.py


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2574k  100 2574k    0     0  9670k      0 --:--:-- --:--:-- --:--:-- 9641k


In [ ]:
!python3 get-pip.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
!python3 -m pip install --upgrade pip


In [ ]:
!python3 -m pip install torch==1.8.0 torchvision==0.9.0
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


ERROR: Could not find a version that satisfies the requirement torch==1.8.0 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0)
ERROR: No matching distribution found for torch==1.8.0
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ivf73szb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ivf73szb
  Resolved https://github.com/facebookresearch/detectron2.git to commit 536dc9d527074e3b15df5f6677ffe1f4e104a4ab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s

In [ ]:
import os
import json
import shutil
import random
import torch
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.logger import setup_logger
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
setup_logger()

# === STEP 1: Dataset Preparation ===

base_folder = "/content/drive/MyDrive/Colab Notebooks/Labeled"
output_base_dataset_1 = "/content/drive/MyDrive/output_dataset_1"
output_base_dataset_2 = "/content/drive/MyDrive/output_dataset_2"

class_mapping_dataset_1 = {"good": 0, "no_good": 1}
class_mapping_dataset_2 = {"good": 0, "exc_solder": 1, "poor_solder": 2, "spike": 3}

for base in [output_base_dataset_1, output_base_dataset_2]:
    for split in ["train", "val"]:
        os.makedirs(os.path.join(base, "images", split), exist_ok=True)
        os.makedirs(os.path.join(base, "labels", split), exist_ok=True)

train_ratio = 0.8
json_files = [f for f in os.listdir(base_folder) if f.endswith(".json")]
train_files = random.sample(json_files, int(len(json_files) * train_ratio))
val_files = [f for f in json_files if f not in train_files]

def is_valid_for_dataset(json_path, class_mapping):
    with open(json_path, 'r') as file:
        data = json.load(file)
    for shape in data['shapes']:
        if shape['label'] not in class_mapping:
            return False
    return True

for file_set, split in [(train_files, "train"), (val_files, "val")]:
    for filename in file_set:
        json_path = os.path.join(base_folder, filename)
        image_name = os.path.splitext(filename)[0] + ".jpg"
        image_path = os.path.join(base_folder, image_name)

        if not os.path.exists(image_path):
            continue

        # Dataset 1
        if is_valid_for_dataset(json_path, class_mapping_dataset_1):
            dest_img = os.path.join(output_base_dataset_1, "images", split, image_name)
            dest_json = os.path.join(output_base_dataset_1, "labels", split, filename)
            shutil.copy(image_path, dest_img)
            shutil.copy(json_path, dest_json)

        # Dataset 2
        if is_valid_for_dataset(json_path, class_mapping_dataset_2):
            dest_img = os.path.join(output_base_dataset_2, "images", split, image_name)
            dest_json = os.path.join(output_base_dataset_2, "labels", split, filename)
            shutil.copy(image_path, dest_img)
            shutil.copy(json_path, dest_json)

# === STEP 2: Dataset Registration for Detectron2 ===

def load_soldef_detectron2(img_dir, label_dir, class_names):
    dataset_dicts = []
    img_files = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png"))]

    for idx, img_file in enumerate(img_files):
        img_path = os.path.join(img_dir, img_file)
        json_name = os.path.splitext(img_file)[0] + ".json"
        json_path = os.path.join(label_dir, json_name)

        if not os.path.exists(json_path):
            continue

        with open(json_path, 'r') as f:
            data = json.load(f)

        record = {
            "file_name": img_path,
            "image_id": idx,
            "height": data.get("imageHeight"),
            "width": data.get("imageWidth"),
            "annotations": []
        }

        for shape in data.get("shapes", []):
            label = shape["label"]
            if label not in class_names:
                continue
            pts = shape["points"]
            poly = [float(p) for x, y in pts for p in (x, y)]
            xs = [p[0] for p in pts]
            ys = [p[1] for p in pts]
            bbox = [min(xs), min(ys), max(xs), max(ys)]

            record["annotations"].append({
                "bbox": bbox,
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": class_names.index(label)
            })

        dataset_dicts.append(record)
    return dataset_dicts

def register_soldef_dataset(dataset_name, base_path, class_names):
    for split in ["train", "val"]:
        img_dir = os.path.join(base_path, "images", split)
        label_dir = os.path.join(base_path, "labels", split)
        name = f"{dataset_name}_{split}"
        DatasetCatalog.register(name, lambda img=img_dir, lab=label_dir, cls=class_names:
                                load_soldef_detectron2(img, lab, cls))
        MetadataCatalog.get(name).set(thing_classes=class_names)

register_soldef_dataset("soldef1", output_base_dataset_1, ["good", "no_good"])
register_soldef_dataset("soldef2", output_base_dataset_2, ["good", "exc_solder", "poor_solder", "spike"])

# === STEP 3: Training + Evaluation ===

def train_detectron2(dataset_name, num_classes, output_dir, max_iter=1500):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (f"{dataset_name}_train",)
    cfg.DATASETS.TEST = (f"{dataset_name}_val",)
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = max_iter
    cfg.SOLVER.STEPS = []
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.OUTPUT_DIR = output_dir
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # === mAP Evaluation ===
    evaluator = COCOEvaluator(f"{dataset_name}_val", cfg, False, output_dir=output_dir)
    val_loader = build_detection_test_loader(cfg, f"{dataset_name}_val")
    metrics = inference_on_dataset(trainer.model, val_loader, evaluator)

    print(f"\n========== mAP Evaluation for {dataset_name} ==========")
    print(metrics)

# Train + evaluate both datasets
train_detectron2("soldef1", num_classes=2,
                 output_dir="/content/drive/MyDrive/output_dataset_1")

train_detectron2("soldef2", num_classes=4,
                 output_dir="/content/drive/MyDrive/output_dataset_2")


[05/12 18:04:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 121MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[05/12 18:05:22 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/12 18:05:38 d2.utils.events]:  eta: 0:13:25  iter: 19  total_loss: 2.142  loss_cls: 1.004  loss_box_reg: 0.4561  loss_mask: 0.6882  loss_rpn_cls: 0.005573  loss_rpn_loc: 0.007049    time: 0.5421  last_time: 0.6243  data_time: 0.1042  last_data_time: 0.0497   lr: 4.9953e-06  max_mem: 2579M
[05/12 18:05:53 d2.utils.events]:  eta: 0:13:14  iter: 39  total_loss: 2.091  loss_cls: 0.9122  loss_box_reg: 0.4932  loss_mask: 0.6803  loss_rpn_cls: 0.005267  loss_rpn_loc: 0.006741    time: 0.5467  last_time: 0.5124  data_time: 0.0090  last_data_time: 0.0101   lr: 9.9902e-06  max_mem: 2579M
[05/12 18:06:03 d2.utils.events]:  eta: 0:12:40  iter: 59  total_loss: 1.911  loss_cls: 0.737  loss_box_reg: 0.4834  loss_mask: 0.6634  loss_rpn_cls: 0.003947  loss_rpn_loc: 0.007088    time: 0.5357  last_time: 0.5037  data_time: 0.0170  last_data_time: 0.0082   lr: 1.4985e-05  max_mem: 2579M
[05/12 18:06:14 d2.utils.events]:  eta: 0:12:31  iter: 79  total_loss: 1.673  loss_cls: 0.5752  loss_box_reg: 0.4875 

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[05/12 18:20:47 d2.engine.train_loop]: Starting training from iteration 0
[05/12 18:20:59 d2.utils.events]:  eta: 0:14:02  iter: 19  total_loss: 2.986  loss_cls: 1.568  loss_box_reg: 0.6722  loss_mask: 0.6899  loss_rpn_cls: 0.03454  loss_rpn_loc: 0.01597    time: 0.5729  last_time: 0.6959  data_time: 0.0645  last_data_time: 0.0894   lr: 4.9953e-06  max_mem: 2581M
[05/12 18:21:11 d2.utils.events]:  eta: 0:14:00  iter: 39  total_loss: 2.88  loss_cls: 1.436  loss_box_reg: 0.7308  loss_mask: 0.6918  loss_rpn_cls: 0.0196  loss_rpn_loc: 0.01345    time: 0.5805  last_time: 0.5561  data_time: 0.0078  last_data_time: 0.0060   lr: 9.9902e-06  max_mem: 2581M
[05/12 18:21:23 d2.utils.events]:  eta: 0:13:44  iter: 59  total_loss: 2.597  loss_cls: 1.195  loss_box_reg: 0.6687  loss_mask: 0.6858  loss_rpn_cls: 0.02166  loss_rpn_loc: 0.01141    time: 0.5788  last_time: 0.5733  data_time: 0.0147  last_data_time: 0.0092   lr: 1.4985e-05  max_mem: 2581M
[05/12 18:21:34 d2.utils.events]:  eta: 0:13:28  ite

In [ ]:
!python3 -m pip install torch==1.8.0 torchvision==0.9.0
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


ERROR: Could not find a version that satisfies the requirement torch==1.8.0 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0)
ERROR: No matching distribution found for torch==1.8.0
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-3y2zmka5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-3y2zmka5
  Resolved https://github.com/facebookresearch/detectron2.git to commit 536dc9d527074e3b15df5f6677ffe1f4e104a4ab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.0 MB/

In [ ]:
import os
import json
import shutil
import random
import torch
import cv2
import albumentations as A
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.logger import setup_logger
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetMapper

setup_logger()
random.seed(42)

# === STEP 1: Dataset Preparation ===

base_folder = "/content/drive/MyDrive/Colab Notebooks/Labeled"
output_base_dataset_1 = "/content/drive/MyDrive/output_dataset_1"
output_base_dataset_2 = "/content/drive/MyDrive/output_dataset_2"

class_mapping_dataset_1 = {"good": 0, "no_good": 1}
class_mapping_dataset_2 = {"good": 0, "exc_solder": 1, "poor_solder": 2, "spike": 3}

for base in [output_base_dataset_1, output_base_dataset_2]:
    for split in ["train", "val"]:
        os.makedirs(os.path.join(base, "images", split), exist_ok=True)
        os.makedirs(os.path.join(base, "labels", split), exist_ok=True)

train_ratio = 0.8
json_files = [f for f in os.listdir(base_folder) if f.endswith(".json")]
train_files = random.sample(json_files, int(len(json_files) * train_ratio))
val_files = [f for f in json_files if f not in train_files]

def is_valid_for_dataset(json_path, class_mapping):
    with open(json_path, 'r') as file:
        data = json.load(file)
    for shape in data['shapes']:
        if shape['label'] not in class_mapping:
            return False
    return True

for file_set, split in [(train_files, "train"), (val_files, "val")]:
    for filename in file_set:
        json_path = os.path.join(base_folder, filename)
        image_name = os.path.splitext(filename)[0] + ".jpg"
        image_path = os.path.join(base_folder, image_name)

        if not os.path.exists(image_path):
            continue

        # Dataset 1
        if is_valid_for_dataset(json_path, class_mapping_dataset_1):
            shutil.copy(image_path, os.path.join(output_base_dataset_1, "images", split, image_name))
            shutil.copy(json_path, os.path.join(output_base_dataset_1, "labels", split, filename))

        # Dataset 2
        if is_valid_for_dataset(json_path, class_mapping_dataset_2):
            shutil.copy(image_path, os.path.join(output_base_dataset_2, "images", split, image_name))
            shutil.copy(json_path, os.path.join(output_base_dataset_2, "labels", split, filename))

# === STEP 2: Dataset Registration ===

def load_soldef_detectron2(img_dir, label_dir, class_names):
    dataset_dicts = []
    img_files = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png"))]

    for idx, img_file in enumerate(img_files):
        img_path = os.path.join(img_dir, img_file)
        json_name = os.path.splitext(img_file)[0] + ".json"
        json_path = os.path.join(label_dir, json_name)

        if not os.path.exists(json_path):
            continue

        with open(json_path, 'r') as f:
            data = json.load(f)

        record = {
            "file_name": img_path,
            "image_id": idx,
            "height": data.get("imageHeight"),
            "width": data.get("imageWidth"),
            "annotations": []
        }

        for shape in data.get("shapes", []):
            label = shape["label"]
            if label not in class_names:
                continue
            pts = shape["points"]
            poly = [float(p) for x, y in pts for p in (x, y)]
            xs = [p[0] for p in pts]
            ys = [p[1] for p in pts]
            bbox = [min(xs), min(ys), max(xs), max(ys)]

            record["annotations"].append({
                "bbox": bbox,
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": class_names.index(label)
            })

        dataset_dicts.append(record)
    return dataset_dicts

def register_soldef_dataset(dataset_name, base_path, class_names):
    for split in ["train", "val"]:
        img_dir = os.path.join(base_path, "images", split)
        label_dir = os.path.join(base_path, "labels", split)
        name = f"{dataset_name}_{split}"
        DatasetCatalog.register(name, lambda img=img_dir, lab=label_dir, cls=class_names:
                                load_soldef_detectron2(img, lab, cls))
        MetadataCatalog.get(name).set(thing_classes=class_names)

register_soldef_dataset("soldef1", output_base_dataset_1, ["good", "no_good"])
register_soldef_dataset("soldef2", output_base_dataset_2, ["good", "exc_solder", "poor_solder", "spike"])

# === STEP 3: Albumentations Mapper ===

class AlbumentationsMapper(DatasetMapper):
    def __init__(self, cfg, is_train=True):
        super().__init__(cfg, is_train)
        self.augmentations = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5),
            A.RandomBrightnessContrast(p=0.4),
            A.MotionBlur(p=0.2),
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

    def __call__(self, dataset_dict):
        dataset_dict = dataset_dict.copy()
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
        annos = dataset_dict["annotations"]

        bboxes = [anno["bbox"] for anno in annos]
        category_ids = [anno["category_id"] for anno in annos]

        transformed = self.augmentations(image=image, bboxes=bboxes, category_ids=category_ids)
        image = transformed["image"]
        annos = transformed["bboxes"]
        cats = transformed["category_ids"]

        dataset_dict["image"] = torch.from_numpy(image.transpose(2, 0, 1)).float()
        new_annos = []
        for bbox, category_id in zip(annos, cats):
            x0, y0, x1, y1 = bbox
            new_annos.append({
                "bbox": [x0, y0, x1, y1],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": category_id,
            })
        dataset_dict["instances"] = utils.annotations_to_instances(new_annos, image.shape[:2])
        return dataset_dict

# === STEP 4: Custom Trainer with Albumentations ===

class TrainerWithAlbumentations(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=AlbumentationsMapper(cfg, is_train=True))

# === STEP 5: Training + Evaluation ===

def train_detectron2(dataset_name, num_classes, output_dir, max_iter=1500):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (f"{dataset_name}_train",)
    cfg.DATASETS.TEST = (f"{dataset_name}_val",)
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = max_iter
    cfg.SOLVER.STEPS = []
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    cfg.OUTPUT_DIR = output_dir
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    trainer = TrainerWithAlbumentations(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # mAP Evaluation
    evaluator = COCOEvaluator(f"{dataset_name}_val", cfg, False, output_dir=output_dir)
    val_loader = build_detection_test_loader(cfg, f"{dataset_name}_val")
    metrics = inference_on_dataset(trainer.model, val_loader, evaluator)
    print(f"\n========== mAP Evaluation for {dataset_name} ==========")
    print(metrics)

# === STEP 6: Train Both Datasets ===

train_detectron2("soldef1", num_classes=2, output_dir="/content/drive/MyDrive/output_dataset_1")
train_detectron2("soldef2", num_classes=4, output_dir="/content/drive/MyDrive/output_dataset_2")


[05/13 05:39:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 131MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[05/13 05:40:12 d2.engine.train_loop]: Starting training from iteration 0


<ipython-input-3-7534384d1617>:151: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  dataset_dict["image"] = torch.from_numpy(image.transpose(2, 0, 1)).float()
<ipython-input-3-7534384d1617>:151: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  dataset

ERROR [05/13 05:40:15 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/detectron2/engine/train_loop.py", line 155, in train
    self.run_step()
  File "/usr/local/lib/python3.11/dist-packages/detectron2/engine/defaults.py", line 530, in run_step
    self._trainer.run_step()
  File "/usr/local/lib/python3.11/dist-packages/detectron2/engine/train_loop.py", line 310, in run_step
    loss_dict = self.model(data)
                ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/detectron2/modeling/meta_arch/rcnn.py", l

AttributeError: Cannot find field 'gt_masks' in the given Instances!